In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
@author: marcodia
"""
import numpy as np
import random
import xarray as xr
import pandas as pd
import datetime as dt
import time
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import preprocessing
import tensorflow as tf

import import_ipynb
import sys
import os 

!ln -s /Users/marcodia/Research/Kirsten_code/network_arch.ipynb network_arch.ipynb
!ln -s /Users/marcodia/Research/Kirsten_code/metrics.ipynb metrics.ipynb
!ln -s /Users/marcodia/Research/Kirsten_code/plot.ipynb plot.ipynb
!ln -s /Users/marcodia/Research/Kirsten_code/settings.ipynb settings.ipynb

import network_arch as network
import metrics
import plot
import settings

import cftime
import matplotlib.pyplot as plt
import nc_time_axis

In [ ]:
#%% >>>>> ADDITIONAL FUNCTIONS >>>>>
def is_ndjf(month):
    return np.logical_or(month<=2, month>=11)

def is_ndjfm(month):
    return np.logical_or(month<=3, month>=11)

#%% >>>>> NETWORK SETUP >>>>>
# MAKE THE NN ARCHITECTURE
def make_model():
    # Define and train the model
    tf.keras.backend.clear_session()
    
    # NORMAL ANN
    model = network.defineNN(HIDDENS,
                             input1_shape = X_train.shape[1],
                             output_shape=NLABEL,
                             ridge_penalty1=RIDGE1,
                             dropout=DROPOUT,
                             act_fun='relu',
                             network_seed=NETWORK_SEED)
    
    loss_function = tf.keras.losses.CategoricalCrossentropy()
    #loss_function = tf.keras.losses.SparseCategoricalCrossentropy()
    
    model.compile(
                  optimizer = tf.keras.optimizers.Adam(learning_rate=LR_INIT),
                  loss = loss_function,
                  metrics = [
                      tf.keras.metrics.CategoricalAccuracy(name="categorical_accuracy", dtype=None),
                      metrics.PredictionAccuracy(NLABEL)
                      # in the future, remove metrics.PredictionAccuracy(NLABEL) from above, 
                      # & use "val_categorical_accuracy" (not "val_prediction_accuracy")
                      # in early stopping code (below) and plot.py code
                      # ...then you dont need to import metrics!
                      # "metrics.PredictionAccuracy(NLABEL)" was left over
                      # from abstention code since it removes those that were abstained on
                      # to calculate accuracy.
                      ]
                  )           
    
    #model.summary()    
    return model, loss_function

#---------------------------------------------------
#LEARNING RATE CALLBACK FUNCTION
def scheduler(epoch, lr):
    # This function keeps the initial learning rate for the first ten epochs
    # and decreases it exponentially after that.
    if epoch < 10:
        return lr
    else:
        return lr * tf.math.exp(-0.1)
#---------------------------------------------------

In [ ]:
num_experiments = 10
num_seeds = 10

In [ ]:
#-- Pull Parameters from settings.ipynb ------
for sub_exp in np.arange(101,102):
    EXPERIMENT = 'exp_1/exp_'+str(sub_exp)

    Xdata_folder = 'exp_1/exp_101'
    Ydata_folder = 'exp_1/exp_101'

    ddir_X = '/Users/marcodia/Research/Data/processed_fields/Week_34/'+Xdata_folder+'/'
    ddir_Y = '/Users/marcodia/Research/Data/processed_fields/Week_34/'+Ydata_folder+'/'

    #ddir = '/Users/marcodia/Research/Data/processed_fields/'+EXPERIMENT+'/'
    ddir_out = '/Users/marcodia/Research/Data/processed_fields/Week_45/exp_1/untrained_model/'
    params = settings.get_settings(EXPERIMENT)

    PREDICTOR_VAR  = params['PREDICTOR_VAR']           
    PREDICTAND_VAR = params['PREDICTAND_VAR']              
    REGION_TOR     = params['REGION_TOR']          
    REGION_TAND    = params['REGION_TAND']            
    training_ens   = params['training_ens']            
    validation_ens = params['validation_ens']           
    testing_ens    = params['testing_ens']           
    train_list     = params['train_list']           
    lead           = 21 #params['lead']            
    days_average   = params['days_average']            
    GLOBAL_SEED    = params['GLOBAL_SEED']            
    HIDDENS        = params['HIDDENS']          
    DROPOUT        = params['DROPOUT']            
    RIDGE1         = params['RIDGE1']                    
    LR_INIT        = params['LR_INIT']
    BATCH_SIZE     = params['BATCH_SIZE']           
    RANDOM_SEED    = params['RANDOM_SEED']            
    act_fun        = params['act_fun']            
    N_EPOCHS       = params['N_EPOCHS']           
    PATIENCE       = params['PATIENCE']   
    
#>>>>>SET UP <<<<<<<<<<<<<<<
    np.random.seed(GLOBAL_SEED)
    random.seed(GLOBAL_SEED)
    tf.compat.v1.random.set_random_seed(GLOBAL_SEED)

    NLABEL = 2

    YEARS = '1850-1949'
    STRT = pd.to_datetime('11-01-1850')
    END   = pd.to_datetime('2-28-1949')  + dt.timedelta(days=1)

    time_range = xr.cftime_range(str(STRT)[:10], str(END)[:10],calendar = 'noleap') #[0:10] corresponds to full datestamp
    time_range_ndjf = time_range.where(is_ndjf(time_range.month)).dropna()
    TIME_X = xr.DataArray(time_range_ndjf + dt.timedelta(days=0), dims=['time'])     
    TIME_Y = xr.DataArray(time_range_ndjf + dt.timedelta(days=lead+days_average), dims=['time'])  #below comment explains time segmentation

        #Based on the 14-day running average, Dec 6 1850 corresponds to the prior 14days averaged together. 
        #So a Nov 1 1850 prediction is for 21 days later of the 3-4week future average
    
    # ----- X TRAINING ------
    X_finame = PREDICTOR_VAR+'_'+REGION_TOR+'_'+YEARS+'_'+'ens'+train_list+'_dailyanom_detrend.nc'
    X_all_full = xr.open_dataarray(ddir_X+X_finame)
    X_all = X_all_full.where(X_all_full.time == TIME_X, drop=True)


    Xtrain = X_all.stack(time_all=('ens','time')) # lat,lon,time*8 (8= number of training ens members) 
    Xtrain = Xtrain.transpose('time_all','lat','lon') # time*8,lat,lon

    Xtrain_std = np.std(Xtrain,axis=0)
    Xtrain_mean = np.mean(Xtrain,axis=0)
    Xtrain = (Xtrain-Xtrain_mean)/Xtrain_std
    X_train = Xtrain.stack(z=('lat','lon'))

    # ---------- X VALIDATION (and TESTING)----------
    X_finame  = PREDICTOR_VAR+'_'+REGION_TOR+'_'+YEARS+'_'+'ens'+str(validation_ens)+'_dailyanom_detrend.nc'
    Xval = xr.open_dataarray(ddir_X+X_finame)

    Xval= Xval.where(Xval.time == TIME_X, drop=True)
    Xval = (Xval - Xtrain_mean)/Xtrain_std

    X_TEST_finame  = PREDICTOR_VAR+'_'+REGION_TOR+'_'+YEARS+'_'+'ens'+str(testing_ens)+'_dailyanom_detrend.nc'
    Xtest = xr.open_dataarray(ddir_X+X_TEST_finame)

    Xtest = Xtest.where(Xtest.time == TIME_X, drop=True)
    Xtest = (Xtest - Xtrain_mean)/Xtrain_std

    #%% ----- Y TRAINING--------

    Ytrain_finame = PREDICTAND_VAR+'_'+REGION_TAND+'_boxavg_'+YEARS+'_'+'ens'+train_list+'_dailyanom_detrend_14dayavg.nc'
    Y_all = xr.open_dataarray(ddir_Y+Ytrain_finame)
    Y_all= Y_all.where(Y_all.time == TIME_Y, drop=True)

    # ----- Standardize Y training -----
    Ytrain = Y_all[:,:]    #already box averaged so no lat or lon dimension 
    Ytrain = Ytrain.stack(time_all=('ens','time')) # time*8 (Ytrain time is actually 365*100-(13*8) because of 14-day average)
    Ytrain_med = np.median(Ytrain)
    Ytrain = Ytrain - Ytrain_med       #Subtracting the median forces the output above or below zero 

    # ----- Y VALIDATION --------
    Yval_finame = PREDICTAND_VAR+'_'+REGION_TAND+'_boxavg_'+YEARS+'_'+'ens'+str(validation_ens)+'_dailyanom_detrend_14dayavg.nc'
    Yval_all  = xr.open_dataarray(ddir_Y+Yval_finame)
    Yval_all= Yval_all.where(Yval_all.time == TIME_Y, drop=True)

    # ----- Standardize Y validation -----
    Yval = Yval_all[:]
    Yval = Yval - Ytrain_med         

    # ----- Grab Y testing -----

    Ytest_finame = PREDICTAND_VAR+'_'+REGION_TAND+'_boxavg_'+YEARS+'_'+'ens'+str(testing_ens)+'_dailyanom_detrend_14dayavg.nc'
    Ytest_all  = xr.open_dataarray(ddir_Y+Ytest_finame)
    Ytest_all= Ytest_all.where(Ytest_all.time == TIME_Y, drop=True)

    Ytest = Ytest_all[:]
    Ytest = Ytest - Ytrain_med 

    # ----- Make binary -----
    # training
    Ytrain[np.where(Ytrain>=0)[0]] = 1
    Ytrain[np.where(Ytrain<0)[0]]  = 0
    # validation
    Yval[np.where(Yval>=0)[0]] = 1
    Yval[np.where(Yval<0)[0]]  = 0
    # testing
    Ytest[np.where(Ytest>=0)[0]] = 1
    Ytest[np.where(Ytest<0)[0]]  = 0

    # make Yval have equal 0s and 1s so that random chance is 50%
    n_valzero = np.shape(np.where(Yval==0)[0])[0]
    n_valone  = np.shape(np.where(Yval==1)[0])[0]
    i_valzero = np.where(Yval==0)[0]
    i_valone  = np.where(Yval==1)[0]

    if n_valone > n_valzero:
        isubset_valone = np.random.choice(i_valone,size=n_valzero,replace=False)
        i_valnew = np.sort(np.append(i_valzero,isubset_valone))
        Yval = Yval.isel(time=i_valnew,drop=True)
        X_val  = Xval[i_valnew].stack(z=('lat','lon'))
    elif n_valone < n_valzero:
        isubset_valzero = np.random.choice(i_valzero,size=n_valone,replace=False)
        i_valnew = np.sort(np.append(isubset_valzero,i_valone))
        Yval = Yval.isel(time=i_valnew,drop=True)
        X_val  = Xval[i_valnew].stack(z=('lat','lon'))
    else:
        X_val = Xval.stack(z=('lat','lon'))

    # ----- Make one hot vector -----
    enc = preprocessing.OneHotEncoder()
    onehotlabels      = enc.fit_transform(np.array(Ytrain).reshape(-1, 1)).toarray()
    onehotlabels_val  = enc.fit_transform(np.array(Yval).reshape(-1, 1)).toarray()
    onehotlabels_test  = enc.fit_transform(np.array(Ytest).reshape(-1, 1)).toarray()

    X_test_calcs = Xtest.stack(z=('lat','lon'))
    hotlabels_test_calcs = onehotlabels_test[:,:NLABEL]

# DO NOT TRAIN MODEL; we want to see how the model looks with NO training, so just predicting on testing data 
# there is no early stopping callback and no plotting of loss/accuracy because of history variable in plotting code 

    for NETWORK_SEED in RANDOM_SEED:
        print(NETWORK_SEED)
        # the network seed changes the random seed for the initialized weights.
        # this means that a different network seed can give a different result (e.g. it finds a different minimum in the loss)
        # ----- MAKE NN -----
        es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss',     #monitor='val_prediction_accuracy'
                                                       patience=PATIENCE,
                                                       mode='auto',
                                                       restore_best_weights=True,
                                                       verbose=1)
        #lr_callback = tf.keras.callbacks.LearningRateScheduler(scheduler,verbose=0)
        #callbacks = [es_callback,lr_callback]
        #callbacks = [es_callback]
        callbacks = []

        model, loss_function = make_model()

        hotlabels = onehotlabels[:,:model.output_shape[-1]]
        hotlabels_val = onehotlabels_val[:,:model.output_shape[-1]]

        # ----- TRAINING NETWORK -----
        start_time = time.time()
        # history = model.fit(X_train,
        #                     hotlabels,
        #                     validation_data=(X_val, hotlabels_val),
        #                     batch_size=BATCH_SIZE,
        #                     epochs=N_EPOCHS,
        #                     shuffle=True,
        #                     verbose=0,
        #                     callbacks=callbacks,
        #                    )

        history = model.predict(X_test_calcs)
        stop_time = time.time()
        tf.print(f"Elapsed time during fit = {stop_time - start_time:.2f} seconds\n")

        # ----- SAVE MODEL -----
        fi = EXPERIMENT[-3:]+'_operationalseed'+str(NETWORK_SEED)+'.h5' 
        model.save_weights(ddir_out+fi)

        # ----- PLOT THE RESULTS -----
        # plot.plot_results(
        #     history,
        #     exp_info=(N_EPOCHS, HIDDENS, LR_INIT, BATCH_SIZE, NETWORK_SEED, PATIENCE, RIDGE1, DROPOUT),
        #     showplot=True
        # )

        # ----- PRINT THE RESULTS -----
        predictions = np.argmax(model.predict(X_val),axis=-1)
        predictions_training = np.argmax(model.predict(X_train),axis=-1)
        confusion_training = tf.math.confusion_matrix(labels=Ytrain, predictions=predictions_training)
        confusion = tf.math.confusion_matrix(labels=Yval, predictions=predictions)

        # PRECISION = correct predictions of a class / total predictions for that class
        zero_precision  = (np.sum(confusion[0,0])/np.sum(confusion[:,0])) * 100
        one_precision   = (np.sum(confusion[1,1])/np.sum(confusion[:,1])) * 100

        # Number of times network predicts a given class
        zero_predictions  = (np.shape(np.where(predictions==0))[1]/predictions.shape[0])* 100
        one_predictions   = (np.shape(np.where(predictions==1))[1]/predictions.shape[0])* 100

        print('Zero prediction accuracy: '+str(zero_precision)[:2]+'%')
        print('Zero: '+str(zero_predictions)[:3]+'% of predictions')
        print('One prediction accuracy: '+str(one_precision)[:2]+'%')
        print('One: '+str(one_predictions)[:3]+'% of predictions')

        #print('Validation Loss at Best Epoch: '+str(es_callback.best*1))#+'%')

        # ----- END NETWORK LOOP -----

    NETWORK_SEED=0
    model, LOSS = make_model() 

    tf.get_logger().setLevel('ERROR')
    #---Testing Calcs 
    
    running_window_yr = 10 #running mean
    days_per_annualszn = 30+31+31+28
    running_window = running_window_yr * days_per_annualszn

    array_size = 100 - running_window_yr#-2   #there are less days in Cpval_pred and Ctval_true because of the week 3-4 averaging 
    #accuracy_test = np.zeros([len(RANDOM_SEED),array_size])
    accuracy_test = np.zeros([array_size])
    timeplot = np.arange(1850,1950)
    timeplot = timeplot[running_window_yr:(len(timeplot))]
    
    fig, (ax1) = plt.subplots(1, figsize=(12,6))
    
    stored_accuracy = np.zeros(len(RANDOM_SEED))
    ind = 0
    ind2 = 0
    
    for SEED in RANDOM_SEED:
        if ind < num_experiments:
            model.load_weights(ddir_out+EXPERIMENT[-3:]+'_operationalseed'+str(SEED)+'.h5')
            #---Testing Calcs 
            Ptest = model.predict(X_test_calcs)    #predicted values on validation - softmax output of confidence 
            Cptest_pred = Ptest.argmax(axis=1)     #0,1 of predicted  
            Cttest_true = hotlabels_test_calcs.argmax(axis=1) #true values on validation 

            X_test_calcs = Xtest.stack(z=('lat','lon'))
            hotlabels_test_calcs = onehotlabels_test[:,:model.output_shape[-1]]
            results_test = model.evaluate(X_test_calcs,hotlabels_test_calcs,verbose = 2)

            stored_accuracy[ind] = results_test[2] #2 hardcoded to get the accuracy (categorical and prediction accuracy are the same here)

            for i in np.arange(0,len(Cptest_pred)-running_window_yr,(days_per_annualszn)):
                if (running_window+i) <= (len(Cptest_pred)):
                    modelcorr_test = Cptest_pred[i:running_window+i]==Cttest_true[i:running_window+i]
                    nmodelcorr_test = modelcorr_test[modelcorr_test].shape[0]
                    ntest_test = Cttest_true[i:running_window+i].shape[0]
                    index_test = int(i/days_per_annualszn)
                    accuracy_test[index_test] = 100*nmodelcorr_test/ntest_test
        ind = ind+1
            
        accuracy_test_xr = xr.DataArray(accuracy_test)
        filename = 'accuracy_testdata_'+str(running_window_yr)+'yr_runavg_exp'+EXPERIMENT[-3:]+'_seed'+str(SEED)+'.nc'
        accuracy_test_xr.to_netcdf(ddir_out+filename, mode='w',format='NETCDF4')
        #ax1.plot(timeplot, accuracy_test[ind2,:], label='Seed '+str(SEED))
        ind2 = ind2+1

        stored_accuracy_xr = xr.DataArray(stored_accuracy)
        filename2 = 'overall_accuracy_testdata'+'_exp'+EXPERIMENT[-3:]+'_allseeds'+'.nc'
        stored_accuracy_xr.to_netcdf(ddir_out+filename2, mode='w',format='NETCDF4')
        
        
    # ax1.set(ylabel='Accuracy (\%)')
    # ax1.set(xlim=(timeplot[0],timeplot[-1]+1))
    # ax1.set(xticks=(np.arange(timeplot[0],timeplot[-1]+5,step=5)))
    # ax1.set(ylim=(40,70))
    # ax1.set(yticks=(np.arange(40,75, step=5)))
    # #ax1.set(ylim=(min(accuracy_test)-2,max(accuracy_val)+2))
    # #ax1.set(yticks=(np.arange(np.round(min(accuracy_test)-2),max(accuracy_val)+4, step=5)))
    # ax1.set(title='Experiment '+EXPERIMENT[-3:]+(str(running_window_yr)+'-yr Running Average of Accuracy'))
    # ax1.legend(loc='upper left',ncol=2)
    #plt.savefig(ddir_out+'accuracy_testdata_timeseries_'+str(running_window_yr)+'yr_runavg_exp'+EXPERIMENT[-3:]+'_allseeds.png', format='png')

In [ ]:
print('sup')